# 094: Data Transformation Pipelines

## 🎯 Learning Objectives

By the end of this notebook, you will:
- **Understand** pipeline orchestration concepts (DAGs, task dependencies, scheduling)
- **Implement** task-based pipelines with retry logic and error handling
- **Build** directed acyclic graphs (DAGs) for complex workflows
- **Apply** pipeline patterns to post-silicon test data processing
- **Evaluate** execution strategies and optimization techniques

## 📚 What are Data Transformation Pipelines?

Data transformation pipelines are **orchestrated sequences of tasks** that move and transform data from source to destination. They implement workflows as directed acyclic graphs (DAGs) where tasks execute based on dependencies.

**Why Pipelines?**
- ✅ **Reliability**: Automatic retries, checkpointing, failure recovery
- ✅ **Scalability**: Parallel execution of independent tasks
- ✅ **Maintainability**: Modular tasks, clear dependencies, testable components
- ✅ **Visibility**: Monitoring, logging, alerting for each task

## 🏭 Post-Silicon Validation Use Cases

**Intel: Daily Test Data Pipeline ($40M Value)**
- Input: 2000+ test programs/day generating 50TB STDF files
- Pipeline: Extract → Validate → Aggregate → Load warehouse → Generate reports
- Value: 99.9% SLA, 4-hour data freshness for fab decisions

**NVIDIA: Multi-Stage GPU Validation ($35M Value)**
- Input: Wafer test + package test + system test data
- Pipeline: Merge test stages → Correlate failures → ML feature engineering → Dashboard
- Value: 500 concurrent pipelines, 15-minute end-to-end latency

**Qualcomm: Global Test Data Sync ($20M Value)**
- Input: 8 fabs worldwide, 24/7 test operations
- Pipeline: Incremental extract → Deduplicate → Standardize → Centralize → Analyze
- Value: 60% reduced data pipeline maintenance costs

**AMD: Adaptive Test Flow ($15M Value)**
- Input: Real-time parametric test results
- Pipeline: Streaming ingest → Outlier detection → Trigger retest → Update binning → Alert
- Value: 10× faster failure detection (10 min → 1 min)

## 🔄 Data Pipeline Workflow

```mermaid
graph LR
    A[Source Data] --> B[Extract Task]
    B --> C[Validate Task]
    C --> D[Transform Task]
    D --> E[Load Task]
    E --> F[Target System]
    
    B -.->|Retry on Fail| B
    C -.->|Retry on Fail| C
    D -.->|Retry on Fail| D
    E -.->|Retry on Fail| E
    
    G[Scheduler] --> B
    G --> C
    G --> D
    G --> E
    
    style A fill:#e1f5ff
    style F fill:#e1ffe1
    style G fill:#fff4e1
```

## 📊 Learning Path Context

**Prerequisites:**
- 091: ETL Fundamentals (extract-transform-load patterns)
- 092: Apache Spark & PySpark (distributed processing)
- 093: Data Cleaning Advanced (data quality rules)

**Next Steps:**
- 095: Stream Processing Real-Time (Kafka, Flink, streaming pipelines)
- 096: Batch Processing at Scale (massive batch workflows)

---

Let's build production data pipelines! 🚀

## 1. Setup and Imports

In [ ]:
# Note: This notebook demonstrates pipeline concepts
# In production, Airflow/Prefect run as separate services

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from typing import Dict, List, Callable, Optional
from dataclasses import dataclass
import time
import json
from enum import Enum

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ Libraries imported")
print("\n📝 Note: This notebook simulates pipeline orchestration")
print("   Production: Use Apache Airflow or Prefect as standalone services")

### 📝 What's Happening in This Code?

**Purpose:** Import libraries for building pipeline orchestration framework

**Key Points:**
- **Airflow/Prefect**: Not imported here (they run as separate services with web UI)
- **Simulation**: We'll build a simplified orchestrator to demonstrate concepts
- **Production**: Airflow runs on Kubernetes/Docker with scheduler, workers, metadata DB

**Why This Matters:** Understanding orchestration concepts is critical before using Airflow/Prefect.

## 2. Pipeline Building Blocks

In [ ]:
class TaskStatus(Enum):
    """Task execution status"""
    PENDING = "pending"
    RUNNING = "running"
    SUCCESS = "success"
    FAILED = "failed"
    SKIPPED = "skipped"

@dataclass
class TaskResult:
    """Result of task execution"""
    task_id: str
    status: TaskStatus
    data: Optional[any] = None
    error: Optional[str] = None
    duration: float = 0.0
    start_time: Optional[datetime] = None
    end_time: Optional[datetime] = None

class Task:
    """A single task in the pipeline"""
    
    def __init__(self, 
                 task_id: str, 
                 function: Callable,
                 retries: int = 3,
                 retry_delay: int = 60):
        self.task_id = task_id
        self.function = function
        self.retries = retries
        self.retry_delay = retry_delay
        self.dependencies: List['Task'] = []
    
    def set_upstream(self, task: 'Task'):
        """Set task dependency (this task depends on upstream task)"""
        self.dependencies.append(task)
    
    def execute(self, context: Dict) -> TaskResult:
        """Execute task with retry logic"""
        attempt = 0
        start_time = datetime.now()
        
        while attempt < self.retries:
            try:
                print(f"🔄 Executing {self.task_id} (attempt {attempt + 1}/{self.retries})")
                
                result_data = self.function(context)
                end_time = datetime.now()
                
                return TaskResult(
                    task_id=self.task_id,
                    status=TaskStatus.SUCCESS,
                    data=result_data,
                    duration=(end_time - start_time).total_seconds(),
                    start_time=start_time,
                    end_time=end_time
                )
            
            except Exception as e:
                attempt += 1
                if attempt < self.retries:
                    print(f"⚠️  Task {self.task_id} failed: {str(e)}. Retrying in {self.retry_delay}s...")
                    time.sleep(self.retry_delay / 60)  # Shortened for demo
                else:
                    end_time = datetime.now()
                    print(f"❌ Task {self.task_id} failed after {self.retries} attempts")
                    return TaskResult(
                        task_id=self.task_id,
                        status=TaskStatus.FAILED,
                        error=str(e),
                        duration=(end_time - start_time).total_seconds(),
                        start_time=start_time,
                        end_time=end_time
                    )

print("✅ Task class defined")

### 📝 What's Happening in This Code?

**Purpose:** Define Task class with retry logic and dependency tracking

**Key Points:**
- **Task**: Wraps a function with metadata (task_id, retries, dependencies)
- **Retry Logic**: Automatically retry failed tasks (3× default with 60s delay)
- **Dependencies**: `set_upstream()` defines task execution order
- **TaskResult**: Captures execution status, data, errors, timing

**Retry Strategy:**
- **Transient failures**: Network timeouts, API rate limits (retry works)
- **Permanent failures**: Logic errors, missing data (retry fails)
- **Exponential backoff**: Retry delay increases (60s → 120s → 240s)

**Why This Matters:** Retry logic is critical for production pipelines (network failures, API timeouts).

## 3. DAG (Directed Acyclic Graph) Orchestrator

In [ ]:
class DAG:
    """Pipeline DAG (Directed Acyclic Graph)"""
    
    def __init__(self, dag_id: str, schedule: str = "@daily"):
        self.dag_id = dag_id
        self.schedule = schedule
        self.tasks: Dict[str, Task] = {}
    
    def add_task(self, task: Task):
        """Add task to DAG"""
        self.tasks[task.task_id] = task
    
    def _topological_sort(self) -> List[Task]:
        """Sort tasks by dependencies (topological order)"""
        # Build adjacency list
        in_degree = {task_id: 0 for task_id in self.tasks}
        adj_list = {task_id: [] for task_id in self.tasks}
        
        for task_id, task in self.tasks.items():
            for dep in task.dependencies:
                adj_list[dep.task_id].append(task_id)
                in_degree[task_id] += 1
        
        # Kahn's algorithm
        queue = [task_id for task_id, degree in in_degree.items() if degree == 0]
        sorted_tasks = []
        
        while queue:
            task_id = queue.pop(0)
            sorted_tasks.append(self.tasks[task_id])
            
            for neighbor in adj_list[task_id]:
                in_degree[neighbor] -= 1
                if in_degree[neighbor] == 0:
                    queue.append(neighbor)
        
        if len(sorted_tasks) != len(self.tasks):
            raise ValueError("Cycle detected in DAG!")
        
        return sorted_tasks
    
    def execute(self, context: Optional[Dict] = None) -> Dict[str, TaskResult]:
        """Execute all tasks in topological order"""
        if context is None:
            context = {}
        
        context['dag_id'] = self.dag_id
        context['execution_date'] = datetime.now()
        
        print(f"\n{'='*60}")
        print(f"🚀 Executing DAG: {self.dag_id}")
        print(f"📅 Execution Date: {context['execution_date']}")
        print(f"{'='*60}\n")
        
        sorted_tasks = self._topological_sort()
        results = {}
        
        for task in sorted_tasks:
            # Check if dependencies succeeded
            deps_failed = any(
                results[dep.task_id].status == TaskStatus.FAILED 
                for dep in task.dependencies
            )
            
            if deps_failed:
                print(f"⏭️  Skipping {task.task_id} (upstream failed)")
                results[task.task_id] = TaskResult(
                    task_id=task.task_id,
                    status=TaskStatus.SKIPPED,
                    error="Upstream task failed"
                )
            else:
                # Pass upstream results to context
                for dep in task.dependencies:
                    if results[dep.task_id].data is not None:
                        context[dep.task_id] = results[dep.task_id].data
                
                result = task.execute(context)
                results[task.task_id] = result
                
                if result.status == TaskStatus.SUCCESS:
                    print(f"✅ {task.task_id} completed in {result.duration:.2f}s")
                else:
                    print(f"❌ {task.task_id} failed: {result.error}")
        
        self._print_summary(results)
        return results
    
    def _print_summary(self, results: Dict[str, TaskResult]):
        """Print pipeline execution summary"""
        total = len(results)
        success = sum(1 for r in results.values() if r.status == TaskStatus.SUCCESS)
        failed = sum(1 for r in results.values() if r.status == TaskStatus.FAILED)
        skipped = sum(1 for r in results.values() if r.status == TaskStatus.SKIPPED)
        total_duration = sum(r.duration for r in results.values())
        
        print(f"\n{'='*60}")
        print(f"📊 Pipeline Summary")
        print(f"{'='*60}")
        print(f"Total Tasks: {total}")
        print(f"✅ Success: {success}")
        print(f"❌ Failed: {failed}")
        print(f"⏭️  Skipped: {skipped}")
        print(f"⏱️  Total Duration: {total_duration:.2f}s")
        print(f"{'='*60}\n")

print("✅ DAG class defined")

### 📝 What's Happening in This Code?

**Purpose:** Build DAG orchestrator that executes tasks in correct order

**Key Points:**
- **DAG**: Container for tasks with dependency relationships
- **Topological Sort**: Orders tasks so dependencies execute first (Kahn's algorithm)
- **Dependency Check**: Skip downstream tasks if upstream fails
- **Context**: Shared data structure passed between tasks

**Execution Flow:**
1. Sort tasks topologically (resolve dependencies)
2. Execute each task in order
3. If task fails, skip all downstream tasks
4. Pass results via context (task1_result → task2_input)

**Cycle Detection:**
- DAG must be acyclic (no circular dependencies)
- Example invalid: Task A → Task B → Task C → Task A (cycle!)

**Why This Matters:** Topological sort ensures correct execution order. Intel runs 2000+ DAGs daily with complex dependencies.

## 4. Example Pipeline: STDF Processing

In [ ]:
# Define task functions
def extract_stdf_files(context):
    """Extract STDF files from S3"""
    print("  📥 Extracting STDF files from S3...")
    time.sleep(0.5)  # Simulate S3 download
    
    # Simulate extracted data
    data = pd.DataFrame({
        'wafer_id': [f'W{i}' for i in range(100)],
        'test_value': np.random.normal(1.0, 0.1, 100),
        'passed': np.random.choice([True, False], 100, p=[0.95, 0.05])
    })
    
    print(f"  ✅ Extracted {len(data)} records")
    return data

def clean_data(context):
    """Clean and validate data"""
    print("  🧹 Cleaning data...")
    time.sleep(0.3)
    
    # Get data from upstream task
    data = context['extract_stdf']
    
    # Remove outliers (Z-score > 3)
    z_scores = np.abs((data['test_value'] - data['test_value'].mean()) / data['test_value'].std())
    data_clean = data[z_scores < 3]
    
    print(f"  ✅ Cleaned: {len(data)} → {len(data_clean)} records (removed {len(data) - len(data_clean)} outliers)")
    return data_clean

def aggregate_statistics(context):
    """Calculate yield statistics"""
    print("  📊 Aggregating statistics...")
    time.sleep(0.3)
    
    data = context['clean_data']
    
    stats = {
        'total_wafers': len(data['wafer_id'].unique()),
        'total_tests': len(data),
        'yield_pct': (data['passed'].sum() / len(data) * 100),
        'mean_test_value': data['test_value'].mean(),
        'std_test_value': data['test_value'].std()
    }
    
    print(f"  ✅ Yield: {stats['yield_pct']:.1f}%, Mean: {stats['mean_test_value']:.3f}")
    return stats

def load_to_warehouse(context):
    """Load data to data warehouse"""
    print("  💾 Loading to warehouse...")
    time.sleep(0.5)
    
    data = context['clean_data']
    stats = context['aggregate_stats']
    
    # Simulate database write
    print(f"  ✅ Loaded {len(data)} records to warehouse")
    print(f"  📈 Yield: {stats['yield_pct']:.1f}%")
    return {"records_loaded": len(data)}

def send_alert(context):
    """Send alert if yield is low"""
    print("  📧 Checking yield threshold...")
    time.sleep(0.2)
    
    stats = context['aggregate_stats']
    
    if stats['yield_pct'] < 90:
        print(f"  ⚠️  ALERT: Low yield detected ({stats['yield_pct']:.1f}%)")
        print(f"  📧 Sent alert to operations team")
    else:
        print(f"  ✅ Yield OK ({stats['yield_pct']:.1f}%)")
    
    return {"alert_sent": stats['yield_pct'] < 90}

# Build DAG
dag = DAG(dag_id="stdf_processing_pipeline", schedule="@daily")

# Create tasks
extract_task = Task("extract_stdf", extract_stdf_files, retries=3)
clean_task = Task("clean_data", clean_data, retries=2)
aggregate_task = Task("aggregate_stats", aggregate_statistics, retries=1)
load_task = Task("load_warehouse", load_to_warehouse, retries=3)
alert_task = Task("send_alert", send_alert, retries=1)

# Define dependencies
clean_task.set_upstream(extract_task)  # clean depends on extract
aggregate_task.set_upstream(clean_task)  # aggregate depends on clean
load_task.set_upstream(clean_task)  # load depends on clean
alert_task.set_upstream(aggregate_task)  # alert depends on aggregate

# Add tasks to DAG
dag.add_task(extract_task)
dag.add_task(clean_task)
dag.add_task(aggregate_task)
dag.add_task(load_task)
dag.add_task(alert_task)

print("✅ Pipeline DAG built successfully")
print(f"\n📋 DAG: {dag.dag_id}")
print(f"⏰ Schedule: {dag.schedule}")
print(f"📊 Tasks: {len(dag.tasks)}")

### 📝 What's Happening in This Code?

**Purpose:** Build complete STDF processing pipeline with 5 tasks and dependencies

**Key Points:**
- **Task Functions**: Each function is a pipeline stage (extract, clean, aggregate, load, alert)
- **Dependencies**: Clean → Extract, Aggregate → Clean, Load → Clean, Alert → Aggregate
- **Context Passing**: Results flow between tasks (extract_stdf → clean_data → aggregate_stats)
- **Schedule**: `@daily` means run once per day (Airflow cron syntax)

**Dependency Graph:**
```
extract_stdf
     |
  clean_data
    /    \
aggregate  load_warehouse
   |
send_alert
```

**Why This Matters:** Real pipelines have 10-100 tasks with complex dependencies. Intel manages 2000+ daily DAGs.

## 5. Execute Pipeline

In [ ]:
# Execute the pipeline
results = dag.execute()

# Visualize execution timeline
fig, ax = plt.subplots(figsize=(14, 6))

task_names = list(results.keys())
task_durations = [results[task].duration for task in task_names]
task_colors = [
    'green' if results[task].status == TaskStatus.SUCCESS 
    else 'red' if results[task].status == TaskStatus.FAILED
    else 'gray'
    for task in task_names
]

bars = ax.barh(task_names, task_durations, color=task_colors, alpha=0.7)
ax.set_xlabel('Duration (seconds)', fontsize=12, fontweight='bold')
ax.set_ylabel('Task', fontsize=12, fontweight='bold')
ax.set_title('Pipeline Execution Timeline', fontsize=14, fontweight='bold')
ax.grid(axis='x', alpha=0.3)

# Add duration labels
for i, (task, duration) in enumerate(zip(task_names, task_durations)):
    ax.text(duration + 0.02, i, f'{duration:.2f}s', 
            va='center', fontsize=10, fontweight='bold')

# Legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='green', alpha=0.7, label='Success'),
    Patch(facecolor='red', alpha=0.7, label='Failed'),
    Patch(facecolor='gray', alpha=0.7, label='Skipped')
]
ax.legend(handles=legend_elements, loc='lower right')

plt.tight_layout()
plt.show()

print("\n📊 Pipeline execution completed!")

### 📝 What's Happening in This Code?

**Purpose:** Execute pipeline and visualize task execution timeline

**Key Points:**
- **Execution**: DAG runs tasks in topological order
- **Timeline**: Horizontal bar chart shows task durations
- **Status Colors**: Green (success), Red (failed), Gray (skipped)
- **Bottlenecks**: Longest tasks are optimization opportunities

**Production Observability:**
- **Airflow UI**: Gantt chart, task logs, retry history
- **Metrics**: Grafana dashboards (task duration, failure rate, SLA)
- **Alerts**: PagerDuty, Slack (pipeline failures, SLA breaches)
- **Logs**: CloudWatch, ELK stack (debug failed tasks)

**Why This Matters:** Observability is critical for production pipelines. AMD detects failures 10× faster with monitoring ($15M savings).

## 6. Real-World Projects & Business Impact

### 🏭 Post-Silicon Validation Projects

**1. Intel Daily Test Data Pipeline ($40M Annual Impact)**
- **Objective**: Orchestrate 2000+ daily ETL jobs processing 500TB STDF data
- **Data**: Wafer probe + final test from 100+ ATE systems (Oregon, Arizona, Ireland, Israel)
- **Architecture**: Airflow (scheduler) → Spark jobs → Delta Lake → BI dashboards
- **DAG Structure**:
  - 50 parallel extract jobs (one per site/product)
  - 20 transform jobs (cleaning, validation, enrichment)
  - 10 aggregate jobs (yield, correlation, trends)
  - 5 load jobs (Delta Lake, Snowflake, Tableau)
  - 5 ML jobs (yield prediction model retraining)
- **Optimizations**:
  - Task pools (limit concurrency to 100 Spark jobs)
  - SLA monitoring (alert if >4 hour latency)
  - Dynamic DAG generation (from metadata, not hardcoded)
  - Smart retries (exponential backoff, only transient errors)
- **Metrics**: 99.9% SLA (2000+ pipelines/day), <4 hour end-to-end latency
- **Tech Stack**: Airflow 2.7, Kubernetes, Spark, Delta Lake, Databricks
- **Impact**: $40M operational excellence (automated vs manual orchestration)

**2. NVIDIA GPU Test Orchestration ($35M Annual Savings)**
- **Objective**: Orchestrate 500 concurrent jobs processing 100M GPU test records
- **Data**: Voltage, current, frequency, thermal, yield from 10 global test sites
- **Architecture**: Prefect (scheduler) → dask (parallel) → InfluxDB → Grafana
- **Pipeline Structure**:
  - Real-time: 100 streaming jobs (Kafka → Flink → InfluxDB)
  - Batch: 200 daily jobs (S3 → Spark → Delta Lake)
  - ML: 50 model training jobs (hourly retraining)
  - Reporting: 100 dashboard refresh jobs
  - Alerting: 50 anomaly detection jobs
- **Optimizations**:
  - Prefect flow runs (500 concurrent, 10TB/hour throughput)
  - Dynamic task mapping (fan-out 1 → 100 parallel tasks)
  - Cached results (avoid recomputing expensive tasks)
  - Parameterized flows (reuse DAG for multiple products)
- **Metrics**: 500 concurrent jobs, 10TB/hour, <1 hour latency
- **Tech Stack**: Prefect 2.0, Dask, Kafka, Flink, InfluxDB, Grafana
- **Impact**: $35M faster insights (real-time vs daily batch)

**3. Qualcomm Multi-Site Data Sync Pipeline ($20M Annual Savings)**
- **Objective**: Sync wafer probe + final test data across 10 global sites
- **Data**: Probe (Oregon, Austin) + final test (Penang, Shanghai, Taiwan)
- **Architecture**: Airflow → S3 (staging) → Spark → Snowflake → Tableau
- **Challenges**:
  - Time zones: Coordinate jobs across 10 time zones
  - Network: Cross-region S3 transfers (optimize with multipart uploads)
  - Schema evolution: Handle test program changes (backward compatibility)
  - Data quality: Validate before sync (reject bad batches)
- **Implementation**:
  - Airflow external sensors (wait for upstream site data)
  - Cross-region S3 replication (Oregon → Singapore)
  - Schema registry (Confluent Schema Registry for compatibility)
  - Quality gates (Great Expectations validation before load)
- **Metrics**: <4 hour latency (vs 24 hour manual), 99.5% SLA
- **Tech Stack**: Airflow 2.7, S3, Spark, Snowflake, Great Expectations
- **Impact**: $20M savings (60% reduced maintenance, Airflow vs cron)

**4. AMD Yield Model Retraining Pipeline ($15M Annual Savings)**
- **Objective**: Automated ML pipeline for yield prediction model retraining
- **Data**: Daily wafer test data (100GB), model training (4 hours), deployment
- **Architecture**: Airflow → MLflow → Model Registry → Kubernetes deployment
- **Pipeline Stages**:
  1. Data validation (Great Expectations)
  2. Feature engineering (Spark)
  3. Model training (XGBoost, LightGBM, CatBoost)
  4. Model evaluation (hold-out test set)
  5. Model comparison (MLflow, champion/challenger)
  6. Model deployment (Kubernetes, blue-green)
  7. Model monitoring (Evidently AI, drift detection)
- **Optimizations**:
  - Branching (train 3 models in parallel)
  - Short-circuiting (skip deployment if accuracy < threshold)
  - Rollback (revert to previous model if drift detected)
  - A/B testing (deploy to 10% traffic first)
- **Metrics**: 99% automation (vs 50% manual), 10× faster failure detection
- **Tech Stack**: Airflow, MLflow, XGBoost, Kubernetes, Evidently AI
- **Impact**: $15M reduced downtime (automated model updates)

### 🌐 General AI/ML Projects

**5. Uber Driver-Rider Matching Pipeline ($100M Revenue Impact)**
- **Objective**: Real-time pipeline matching 1M rides/hour globally
- **Data**: Driver locations (GPS), rider requests, traffic, pricing
- **Architecture**: Prefect → Kafka → Flink → ML models → Redis
- **Metrics**: <1s matching latency, 1M rides/hour, 99.99% uptime
- **Tech Stack**: Prefect, Kafka, Flink, Redis, ML models (XGBoost)
- **Impact**: $100M revenue (optimal matching reduces wait times)

**6. Netflix Content Recommendation Pipeline ($80M Revenue Increase)**
- **Objective**: Daily pipeline retraining recommendation models on 500M events
- **Data**: User viewing history, ratings, clickstream, content metadata
- **Architecture**: Airflow → Spark → Feature store → ML training → A/B test
- **Metrics**: Daily retraining (500M events), 30% engagement uplift
- **Tech Stack**: Airflow, Spark, Feature Store, TensorFlow, Kubernetes
- **Impact**: $80M revenue (personalized recommendations drive 80% views)

**7. Airbnb Pricing Optimization Pipeline ($60M Revenue Increase)**
- **Objective**: Daily pipeline optimizing listing prices based on demand
- **Data**: Booking history, search demand, seasonality, events, reviews
- **Architecture**: Airflow → Spark → XGBoost → Price API → MySQL
- **Metrics**: Daily price updates (7M listings), 15% booking uplift
- **Tech Stack**: Airflow, Spark, XGBoost, MySQL, Redis cache
- **Impact**: $60M revenue (dynamic pricing optimizes supply/demand)

**8. PayPal Fraud Detection Pipeline ($150M Fraud Prevention)**
- **Objective**: Real-time pipeline scoring 1B transactions/day for fraud
- **Data**: Transaction details, user behavior, merchant risk, device fingerprint
- **Architecture**: Airflow → Kafka → Flink → XGBoost → Block API
- **Metrics**: <50ms scoring latency, 1B TPS, 95% fraud detection
- **Tech Stack**: Airflow, Kafka, Flink, XGBoost, Redis, PostgreSQL
- **Impact**: $150M fraud prevented (real-time scoring blocks fraud)

---

## 🎯 Key Takeaways

**Pipeline Orchestration Concepts:**
1. **DAG**: Directed Acyclic Graph defines task dependencies
2. **Idempotency**: Rerunning pipeline produces same result (critical for retries)
3. **Fault Tolerance**: Retry failed tasks, skip downstream if upstream fails
4. **Observability**: Logs, metrics, alerts for pipeline health

**Business Impact: $500M Total**
- **Post-Silicon**: Intel $40M + NVIDIA $35M + Qualcomm $20M + AMD $15M = **$110M**
- **General**: Uber $100M + Netflix $80M + Airbnb $60M + PayPal $150M = **$390M**

**Airflow vs Prefect:**
- **Airflow**: Mature, large ecosystem, Kubernetes-native (Intel, Qualcomm use)
- **Prefect**: Modern, Python-native, dynamic DAGs (NVIDIA uses)
- **Both**: Production-grade, support retries, SLAs, observability

**Best Practices:**
- ✅ **Idempotency**: Same input → same output (avoid duplicate data)
- ✅ **Small tasks**: 1 task = 1 responsibility (easier debugging)
- ✅ **Parameterization**: Reuse DAGs for multiple products/sites
- ✅ **SLA monitoring**: Alert if pipeline breaches latency target
- ✅ **Dynamic DAGs**: Generate from metadata (avoid hardcoding 2000 DAGs)

**Common Pitfalls:**
- **Long-running tasks**: 10-hour task blocks executors (split into smaller tasks)
- **Too many retries**: 100 retries waste resources (fix root cause)
- **Circular dependencies**: Task A → B → C → A (cycle, DAG invalid)
- **No observability**: Can't debug failures (add logs, metrics, alerts)

**Next Steps:**
- **095**: Stream Processing (real-time pipelines with Kafka, Flink)
- **097**: Data Lake Architecture (storage layer for pipelines)
- **131**: MLOps Fundamentals (ML model deployment pipelines)

---

**🎉 Congratulations!** You've mastered data pipeline orchestration - from DAG design to fault tolerance to production deployment! 🚀